In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from fpdf import FPDF
import os

# Load the data
customers = pd.read_csv('../data/Customers.csv')
products = pd.read_csv('../data/Products.csv')
transactions = pd.read_csv('../data/Transactions.csv')

# Create outputs folder if it doesn't exist
output_dir = '../outputs/'
os.makedirs(output_dir, exist_ok=True)

# Create a PDF class object
class PDF(FPDF):
    def header(self):
        # Only display the header on the first page
        if self.page_no() == 1:
            self.set_font('Arial', 'B', 12)
            self.cell(200, 10, 'Task 1: Exploratory Data Analysis (EDA) Report', ln=True, align='C')
            self.ln(10)  # Line break

    def chapter_title(self, title):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, title, 0, 1)
        self.ln(5)

    def chapter_body(self, body):
        self.set_font('Arial', '', 12)
        self.multi_cell(0, 10, body)
        self.ln(10)

    def add_plot(self, plot_filename):
        self.image(plot_filename, x=10, y=None, w=180)
        self.ln(85)  # Adjust space for plot image

# Create an instance of the PDF class
pdf = PDF()
pdf.add_page()

# Add insights and plots to PDF
# 1. General data summary
pdf.chapter_title('1. General Data Summary')
summary = f"""
Number of Customers: {customers.shape[0]}
Number of Products: {products.shape[0]}
Number of Transactions: {transactions.shape[0]}

Data Summary for Customers:
{customers.describe()}

Data Summary for Products:
{products.describe()}

Data Summary for Transactions:
{transactions.describe()}
"""
pdf.chapter_body(summary)

# 2. Number of unique customers, products, and transactions
num_customers = customers['CustomerID'].nunique()
num_products = products['ProductID'].nunique()
num_transactions = transactions['TransactionID'].nunique()
pdf.chapter_title('2. Unique Counts')
pdf.chapter_body(f"Number of Unique Customers: {num_customers}\n"
                 f"Number of Unique Products: {num_products}\n"
                 f"Number of Transactions: {num_transactions}\n")

# 3. Distribution of product prices
plt.figure(figsize=(10, 6))
sns.histplot(products['Price'], bins=30, kde=True, color='blue')
plt.title('Product Price Distribution')
plt.xlabel('Price')
plt.ylabel('Frequency')
price_plot_path = os.path.join(output_dir, 'price_distribution.png')
plt.savefig(price_plot_path)
plt.close()
pdf.chapter_title('3. Distribution of Product Prices')
pdf.add_plot(price_plot_path)

# 4. Sales by region
sales_by_region = transactions.merge(customers, on='CustomerID').groupby('Region')['TotalValue'].sum().reset_index()
plt.figure(figsize=(10, 6))
sns.barplot(x='Region', y='TotalValue', data=sales_by_region, palette='viridis')
plt.title('Total Sales by Region')
plt.xlabel('Region')
plt.ylabel('Sales')
region_sales_plot_path = os.path.join(output_dir, 'region_sales.png')
plt.savefig(region_sales_plot_path)
plt.close()
pdf.chapter_title('4. Sales by Region')
pdf.add_plot(region_sales_plot_path)

# Save the PDF to the outputs folder
pdf_output_path = os.path.join(output_dir, 'Task1_EDA_Report.pdf')
pdf.output(pdf_output_path)

print(f"PDF report generated: {pdf_output_path}")


IndexError: list index out of range